### Word Prefix Suffix Analysis

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import re
from kneed import KneeLocator
from pathlib import Path
import shutil

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# pre-suffix select
prefix = False  # True, False  word is prefix  example: prefix = True and suffix = False for Turkish word
suffix = True # True, False  word is suffix

# native word select
file_ext = 1000
word_start = 0  # 0  # native word start index
word_end = 1000  # 28  # native word end index


In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/2-1-Word Prefix Suffix Analysis"

#Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def detect_prefix_suffix_word(df, df_column, word_list, prefix_word=True, suffix_word=False):
    '''
    default parameter:\n
    detect_prefix_suffix_word(df, df_column, word_ety_list, prefix_word=True, suffix_word=False)\n
    detect_prefix_suffix_word(df, "word", ["abacus","aba","su"], prefix=True, suffix=True)\n
    df is dataframe. Each word of word_ety_list search in df_column according to prefix_word and suffix_word condition.\n
    prefix_word and suffix_word are not extention. they represents location of word of word_ety_list (word before or after)
    '''    
    df_prefix_suffix_word_result = pd.DataFrame()
    for i in word_list:
        # suffix result
        if prefix_word:
            word_in_word = df[df[f"{df_column}"].str.contains(fr"{i}(?:$)", na=True)]  # string+ext=> word
            word_in_word.insert(0,"search_word",i)
            df_prefix_suffix_word_result = pd.concat([df_prefix_suffix_word_result, word_in_word], axis=0)
        else:
            pass
        # prefix result
        if suffix_word:
            word_in_word = df[df[f"{df_column}"].str.contains(fr"(?:^){i}", na=True)]  # ext+string=> word
            word_in_word.insert(0,"search_word",i)
            df_prefix_suffix_word_result = pd.concat([df_prefix_suffix_word_result, word_in_word], axis=0)
        else:
            pass    
        
    #df_ety_suffix_word_result = df_word_result.sort_values(by="frequency", ascending=False)
    df_prefix_suffix_word_result.drop_duplicates(inplace=True)
    df_prefix_suffix_word_result.reset_index(drop=True, inplace=True)

    return df_prefix_suffix_word_result

In [5]:
def exract_prefix_suffix(df, source_column, target_column):
    '''
    exract_prefix_suffix(df, source_column, target_column):\n
    exract_prefix_suffix(df, "word_pair", "word"):\n
    df is dataframe. word of source_column search in target_column\n
    and exract prefix or suffix. 
    '''
    for i in range(len(df)):
        source_word = df.loc[i,f"{source_column}"]
        target_word = df.loc[i,f"{target_column}"]
        try:
            search_loc = re.search(fr"{source_word}", target_word, re.UNICODE|re.IGNORECASE)
            search_loc_start = search_loc.span()[0]
            search_loc_end = search_loc.span()[1]
            if search_loc_start > 0:
                var1= target_word[0:search_loc_start]
                prefix_suffix = f"{var1}+"
                df.loc[i,"prefix_suffix"] = prefix_suffix                
            else:
                var2 = target_word[search_loc_end:]
                prefix_suffix = f"+{var2}" 
                df.loc[i,"prefix_suffix"] = prefix_suffix 
        except:
            pass
    return df

In [ ]:
def unrelated_value(tex)

#### Word Data

In [6]:
#Pos_Tag = "VERB" # NOUN, VERB (ol, var stem ayrı), ADJ, ADV, NUM, PRON, CCONJ, ADP, AUX 
file_ext = 1000

In [7]:
df_word_raw = pd.read_excel(f"Turkish_{file_ext}_Process.xlsx")
df_word_raw

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,NUM,NaN,bir,bir,bir,a,a,18835735
1,PRON,NaN,bu,bu,bu,this,this,11062659
2,PRON,Q,ne,ne,ne,what,what,8025880
3,CCONJ,NaN,ve,ve,ve,and,and,7766036
4,ADP,NaN,için,için,için,for,for,5484109
...,...,...,...,...,...,...,...,...
995,ADJ,NaN,resmi,resmi,resmi,formal,formal,68287
996,VERB,NaN,veriyor,ver,ver,giving,give,68163
997,NOUN,NaN,okul,okul,oku,school,school,68160
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [8]:
#df_word_raw_select = df_word_raw[df_word_raw["POS1"] == Pos_Tag]
df_word_raw_select = df_word_raw.copy()
df_word_raw_select = df_word_raw_select.drop_duplicates(subset=["word"])
df_word_raw_select

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,NUM,NaN,bir,bir,bir,a,a,18835735
1,PRON,NaN,bu,bu,bu,this,this,11062659
2,PRON,Q,ne,ne,ne,what,what,8025880
3,CCONJ,NaN,ve,ve,ve,and,and,7766036
4,ADP,NaN,için,için,için,for,for,5484109
...,...,...,...,...,...,...,...,...
995,ADJ,NaN,resmi,resmi,resmi,formal,formal,68287
996,VERB,NaN,veriyor,ver,ver,giving,give,68163
997,NOUN,NaN,okul,okul,oku,school,school,68160
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [9]:
word_list = df_word_raw_select["word"].values.tolist()
#word_list

In [10]:
df_prefix_suffix_word = detect_prefix_suffix_word(df_word_raw_select, "word", word_list, prefix_word=prefix, suffix_word=suffix)
df_prefix_suffix_word

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,bir,NUM,NaN,bir,bir,bir,a,a,18835735
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940
4,bir,ADJ,NaN,birkaç,birkaç,birkaç,a few,a few,404462
...,...,...,...,...,...,...,...,...,...
1878,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290
1879,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163
1880,okul,NOUN,NaN,okul,okul,oku,school,school,68160
1881,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [11]:
df_prefix_suffix = exract_prefix_suffix(df_prefix_suffix_word, "search_word", "word")
df_prefix_suffix

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
0,bir,NUM,NaN,bir,bir,bir,a,a,18835735,+
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641,+az
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400,+i
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940,+likte
4,bir,ADJ,NaN,birkaç,birkaç,birkaç,a few,a few,404462,+kaç
...,...,...,...,...,...,...,...,...,...,...
1878,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290,+um
1879,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163,+
1880,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+
1881,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124,+


In [61]:
df_prefix_suffix_select = pd.DataFrame()
for row_num in range(len(df_prefix_suffix)):
    df_var = df_prefix_suffix.iloc[[row_num],]
    if len("".join(df_var["search_word"])) >= len("".join(df_var["stem"])):
        df_prefix_suffix_select = pd.concat([df_prefix_suffix_select,df_var], axis=0)
    else:
        pass
    
df_prefix_suffix_select = df_prefix_suffix_select.reset_index(drop=True)
df_prefix_suffix_select

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
0,bir,NUM,NaN,bir,bir,bir,a,a,18835735,+
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641,+az
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400,+i
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940,+likte
4,bir,NOUN,NaN,birini,biri,bir,one,somebody,259916,+ini
...,...,...,...,...,...,...,...,...,...,...
1619,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290,+um
1620,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163,+
1621,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+
1622,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124,+


##### Noun

In [62]:
df_noun = df_prefix_suffix_select[df_prefix_suffix_select["POS1"] == "NOUN"]
df_noun

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
4,bir,NOUN,NaN,birini,biri,bir,one,somebody,259916,+ini
5,bir,NOUN,NaN,birisi,biri,bir,someone,somebody,182370,+isi
6,bir,NOUN,NaN,birinin,biri,bir,someone,somebody,135905,+inin
7,bir,NOUN,NaN,birine,biri,bir,to someone,somebody,111455,+ine
9,bir,NOUN,NaN,biriyle,biri,bir,with someone,somebody,82104,+iyle
...,...,...,...,...,...,...,...,...,...,...
1609,güneş,NOUN,NaN,güneş,güneş,güneş,sun,sun,68622,+
1616,önceki,NOUN,NaN,önceki,önce,önce,previous,before,68345,+
1617,ifade,NOUN,NaN,ifade,ifade,ifade,expression,expression,68298,+
1621,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+


In [64]:
df_noun[df_noun["word"].str.contains(fr"lar", na=True) | df_noun["word"].str.contains(fr"ler", na=True)]

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
20,ne,NOUN,Q,neler,ne,ne,what,what,397377,+ler
46,o,NOUN,NaN,onlardan,onlar,o,from them,they,82961,+nlardan
84,şey,NOUN,NaN,şeyler,şey,şey,things,thing,649627,+ler
87,şey,NOUN,NaN,şeyleri,şey,şey,things,thing,137646,+leri
134,zaman,NOUN,TIME,zamanlar,zaman,zaman,times,time,89599,+lar
224,adam,NOUN,NaN,adamlar,adam,adam,men,man,85319,+lar
252,gün,NOUN,NaN,günler,gün,gün,days,day,79133,+ler
259,teşekkürler,NOUN,GRE,teşekkürler,teşekkür,teşekkür,thanks,thanks,680203,+
277,gece,NOUN,TIME,geceler,gece,gece,nights,night,129031,+ler
278,şeyler,NOUN,NaN,şeyler,şey,şey,things,thing,649627,+


In [63]:
df_noun[df_noun["prefix_suffix"].str.contains(fr"lar", na=True) | df_noun["prefix_suffix"].str.contains(fr"ler", na=True)]

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
20,ne,NOUN,Q,neler,ne,ne,what,what,397377,+ler
46,o,NOUN,NaN,onlardan,onlar,o,from them,they,82961,+nlardan
84,şey,NOUN,NaN,şeyler,şey,şey,things,thing,649627,+ler
87,şey,NOUN,NaN,şeyleri,şey,şey,things,thing,137646,+leri
134,zaman,NOUN,TIME,zamanlar,zaman,zaman,times,time,89599,+lar
224,adam,NOUN,NaN,adamlar,adam,adam,men,man,85319,+lar
252,gün,NOUN,NaN,günler,gün,gün,days,day,79133,+ler
277,gece,NOUN,TIME,geceler,gece,gece,nights,night,129031,+ler
370,teşekkür,NOUN,GRE,teşekkürler,teşekkür,teşekkür,thanks,thanks,680203,+ler
402,kız,NOUN,NaN,kızlar,kız,kız,girls,girl,98368,+lar


In [12]:
df_test = df_prefix_suffix.sort_values(["search_word","word"],key=lambda x:x.str.len())
df_test

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
74,o,PRON,NaN,o,o,o,he,he,5013838,+
86,o,VERB,NaN,ol,ol,ol,be,be,600750,+l
110,o,NUM,NaN,on,on,on,ten,ten,168880,+n
75,o,PRON,NaN,onu,o,o,him-her-it,he,2486889,+nu
78,o,PRON,NaN,ona,o,o,him,he,1124548,+na
...,...,...,...,...,...,...,...,...,...,...
1598,inanamıyorum,VERB,NEG,inanamıyorum,inan,inan,i cant believe,believe,90208,+
1744,çalışıyorsun,VERB,NaN,çalışıyorsun,çalış,çalış,you work,work,77707,+
1814,hatırlıyorum,VERB,NaN,hatırlıyorum,hatırla,hatır,i remember,remember,72290,+
1830,istiyorsunuz,VERB,NaN,istiyorsunuz,iste,iste,you want,want,71208,+


In [13]:
#df_test.to_excel("test_result.xlsx", index=False)

In [21]:
df_prefix_suffix[(df_prefix_suffix["prefix_suffix"] == "+iyor") | (df_prefix_suffix["prefix_suffix"] == "+ıyor") | (df_prefix_suffix["prefix_suffix"] == "+uyor") | 
(df_prefix_suffix["prefix_suffix"] == "+üyor")]

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
486,gerek,VERB,NaN,gerekiyor,gerek,gerek,needed,necessary,324783,+iyor
500,ol,VERB,NaN,oluyor,ol,ol,is happening,be,503064,+uyor
550,gel,VERB,NaN,geliyor,gel,gel,coming,come,416067,+iyor
580,ver,VERB,NaN,veriyor,ver,ver,giving,give,68163,+iyor
987,yap,VERB,NaN,yapıyor,yap,yap,doing,do it,179712,+ıyor
1579,düşün,VERB,NaN,düşünüyor,düşün,düş,thinking,think,93634,+üyor


In [18]:
df_prefix_suffix[(df_prefix_suffix["prefix_suffix"] == "+ecek") | (df_prefix_suffix["prefix_suffix"] == "+acak")]

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
496,ol,VERB,NaN,olacak,ol,ol,will be,be,786194,+acak
551,gel,NOUN,TIME,gelecek,gelecek,gel,the future,the future,189299,+ecek
993,yap,VERB,NaN,yapacak,yap,yap,will make,do it,115620,+acak


In [23]:
df_prefix_suffix[(df_prefix_suffix["prefix_suffix"] == "+mış") | (df_prefix_suffix["prefix_suffix"] == "+miş") | (df_prefix_suffix["prefix_suffix"] == "+muş") | (df_prefix_suffix["prefix_suffix"] == "+müş") |
(df_prefix_suffix["prefix_suffix"] == "+dı") | (df_prefix_suffix["prefix_suffix"] == "+di") | (df_prefix_suffix["prefix_suffix"] == "+du") | (df_prefix_suffix["prefix_suffix"] == "+dü") | 
(df_prefix_suffix["prefix_suffix"] == "+tı") | (df_prefix_suffix["prefix_suffix"] == "+ti") | (df_prefix_suffix["prefix_suffix"] == "+tu") | (df_prefix_suffix["prefix_suffix"] == "+tü")]

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix
160,de,VERB,NaN,dedi,de,de,said,also,182523,+di
175,var,VERB,NaN,vardı,var,var,there was,there is,451341,+dı
178,var,VERB,NaN,varmış,var,var,there was,there is,118304,+mış
186,değil,VERB,NEG,değildi,değil,değil,it wasn't,not,235756,+di
220,yok,VERB,NEG,yoktu,yok,yok,there was not,there is not,181801,+tu
403,olur,VERB,NaN,olurdu,ol,ol,would be,be,124365,+du
457,et,VERB,NaN,etti,et,et,he,meat,245330,+ti
492,ol,VERB,NaN,oldu,ol,ol,happened,be,1141161,+du
505,ol,VERB,NaN,olmuş,ol,ol,it is ok,be,213607,+muş
549,gel,VERB,NaN,geldi,gel,gel,came,come,464529,+di


In [20]:
df_prefix_suffix.search_word.nunique()

283

In [21]:
df_prefix_suffix.word.nunique()

283

##### Copy Move And Delete

In [39]:
output_file1 = glob.glob(f"Prefix_Suffix_*Result.*")
output_file1

['Turkish_English_5000_Word_Prefix_Suffix_Custom_Result.xlsx']

In [40]:
for k in output_file1:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [41]:
for i in output_file1:
    try:
        os.remove(i)
    except:
        pass

In [42]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_Prefix_Suffix_*.*")
output_file2

['Turkish_English_5000_Word_Prefix_Suffix_All.csv',
 'Turkish_English_5000_Word_Prefix_Suffix_Select.xlsx']

In [43]:
for l in output_file2:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [44]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

### Concat Native And Etymology Prefix Suffix Result

In [3]:
import os
import pandas as pd
import numpy as np
import glob

In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian, Intersect ==> native language

# file extention
file_ext = 1000  # native word number

In [5]:
def lower_strip(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out

In [6]:
native_file = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{file_ext}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
native_file

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish English/Turkish_English_200_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx']

In [7]:
df_native = pd.read_excel(native_file[0])
df_native = df_native[["search_word","word"]]
df_native

,search_word,word
0,adam,adam
1,adam,adama
2,adam,adamdan
3,adam,adamdı
4,adam,adamdır
...,...,...
1618,şey,şeyler
1619,şey,şeylerden
1620,şey,şeylere
1621,şey,şeyleri


In [8]:
etymology_file = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
etymology_file

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish English/Turkish_English_Shared_Word_Prefix_Suffix_Custom_Result.xlsx']

In [9]:
df_etmology = pd.read_excel(etymology_file[0])
df_etmology = df_etmology[["search_word","word"]]
df_etmology

,search_word,word
0,abaküs,abaküs
1,abandone,abandoned
2,abanoz,abanoz
3,abdomen,abdomende
4,abluka,abluka
...,...,...
6188,şut,şutu
6189,şut,şutunu
6190,şırınga,şırınga
6191,şırınga,şırıngayla


In [10]:
df_native_etymology_concat = pd.concat([df_native,df_etmology], axis=0)
df_native_etymology_concat

,search_word,word
0,adam,adam
1,adam,adama
2,adam,adamdan
3,adam,adamdı
4,adam,adamdır
...,...,...
6188,şut,şutu
6189,şut,şutunu
6190,şırınga,şırınga
6191,şırınga,şırıngayla


In [11]:
df_native_etymology_concat["search_word"] = df_native_etymology_concat["search_word"].apply(lambda x : lower_strip(x))
df_native_etymology_concat["word"] = df_native_etymology_concat["word"].apply(lambda x : lower_strip(x))
df_native_etymology_concat.drop_duplicates(inplace=True)
df_native_etymology_concat.reset_index(drop=True, inplace=True)
df_native_etymology_concat

,search_word,word
0,adam,adam
1,adam,adama
2,adam,adamdan
3,adam,adamdı
4,adam,adamdır
...,...,...
7811,şut,şutu
7812,şut,şutunu
7813,şırınga,şırınga
7814,şırınga,şırıngayla


In [12]:
df_native_etymology_concat.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} \
{lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{file_ext}_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx", index=False)

### Temp

In [ ]:
# English, French, German, Spanish, Portuguese, Dutch, Italian

In [29]:
#df_pair1 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/\
#Turkish English/Turkish_English_Shared_Vocabulary.xlsx")
#df_pair1

In [31]:
#df_pair2 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/\
#Turkish French/Turkish_French_Shared_Vocabulary.xlsx")
#df_pair2

In [32]:
#df_pair3 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/\
#Turkish German/Turkish_German_Shared_Vocabulary.xlsx")
#df_pair3

In [33]:
#df_pair4 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/\
#Turkish Spanish/Turkish_Spanish_Shared_Vocabulary.xlsx")
#df_pair4

In [34]:
#df_pair5 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/\
#Turkish Portuguese/Turkish_Portuguese_Shared_Vocabulary.xlsx")
#df_pair5

In [35]:
#df_pair6 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/\
#Turkish Dutch/Turkish_Dutch_Shared_Vocabulary.xlsx")
#df_pair6

In [36]:
#df_pair7 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/\
#Turkish Italian/Turkish_Italian_Shared_Vocabulary.xlsx")
#df_pair7

In [ ]:
#set1 = set(df_pair1.dict_entry_main)
#set2 = set(df_pair2.dict_entry_main)
#set3 = set(df_pair3.dict_entry_main)
#set4 = set(df_pair4.dict_entry_main)
#set5 = set(df_pair5.dict_entry_main)
#set6 = set(df_pair6.dict_entry_main)
#set7 = set(df_pair7.dict_entry_main)

In [ ]:
#df_ety_intersect = pd.DataFrame((((((set7.intersection(set6)).intersection(set5)).intersection(set4)).intersection(set3)).intersection(set2)).intersection(set1), columns=["dict_entry_main"])
#df_ety_intersect

In [ ]:
#df_ety_intersect.to_excel("Turkish_Intersect_Shared_Vocabulary.xlsx", index=False)

In [19]:
#df_intersect = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish Intersect/Turkish_Intersect_Shared_Vocabulary.xlsx")
#df_intersect  # English, French, German, Spanish, Portuguese, Dutch, Italian

In [20]:
#df_pair1_merge = pd.merge(df_intersect,df_pair1, how="left", on="dict_entry_main")
#df_pair1_merge.drop_duplicates(inplace=True)
#df_pair1_merge

In [21]:
#df_pair2_merge = pd.merge(df_pair1_merge,df_pair2, how="left", on="dict_entry_main")
#df_pair2_merge.drop_duplicates(inplace=True)
#df_pair2_merge

In [22]:
#df_pair3_merge = pd.merge(df_pair2_merge,df_pair3, how="left", on="dict_entry_main")
#df_pair3_merge.drop_duplicates(inplace=True)
#df_pair3_merge

In [23]:
#df_pair4_merge = pd.merge(df_pair3_merge,df_pair4, how="left", on="dict_entry_main")
#df_pair4_merge.drop_duplicates(inplace=True)
#df_pair4_merge

In [24]:
#df_pair5_merge = pd.merge(df_pair4_merge,df_pair5, how="left", on="dict_entry_main")
#df_pair5_merge.drop_duplicates(inplace=True)
#df_pair5_merge

In [25]:
#df_pair6_merge = pd.merge(df_pair5_merge,df_pair6, how="left", on="dict_entry_main")
#df_pair6_merge.drop_duplicates(inplace=True)
#df_pair6_merge

In [26]:
#df_pair7_merge = pd.merge(df_pair6_merge,df_pair7, how="left", on="dict_entry_main")
#df_pair7_merge.drop_duplicates(inplace=True)
#df_pair7_merge

In [27]:
#df_pair7_merge.rename(columns={"dict_entry_main":"turkish_word_intersect"}, inplace=True)
#df_pair7_merge

In [28]:
#df_pair7_merge.to_excel("Turkish_Intersect_Shared_Vocabulary_With_Other_Languages.xlsx", index=False)